Задание 1: Три потока для обработки списка чисел

In [2]:
# Задание 1: Три потока для обработки списка чисел
import threading
import random
import time

# Глобальные переменные и объекты синхронизации
data_list = []
list_filled_event = threading.Event() # Событие для сигнализации о заполнении списка
lock = threading.Lock() # Блокировка для безопасного доступа к data_list (хотя здесь не строго нужна для чтения после заполнения)

# --- Функции для потоков ---
def fill_list(count: int, min_val: int, max_val: int):
    """
    Первый поток: заполняет список случайными числами.
    """
    global data_list
    print("[Поток 1] Начало заполнения списка...")
    for _ in range(count):
        data_list.append(random.randint(min_val, max_val))
        time.sleep(0.01) # Небольшая имитация работы
    print(f"[Поток 1] Список заполнен {len(data_list)} случайными числами.")
    list_filled_event.set() # Сигнализируем, что список заполнен

def calculate_sum():
    """
    Второй поток (в задании указано как "первый" из двух ожидающих):
    ожидает заполнения и вычисляет сумму элементов.
    """
    print("[Поток 2] Ожидание заполнения списка для расчета суммы...")
    list_filled_event.wait() # Ждем события

    # Блокировка здесь не обязательна, если список больше не изменяется после события,
    # но для демонстрации безопасного доступа можно использовать:
    # with lock:
    #     local_list_copy = list(data_list) # Работаем с копией для избежания гонок, если бы список еще мог меняться
    # current_sum = sum(local_list_copy)

    current_sum = sum(data_list) # Список уже не должен меняться
    print(f"[Поток 2] Сумма элементов списка: {current_sum}")
    return current_sum

def calculate_average():
    """
    Третий поток (в задании указано как "второй" из двух ожидающих):
    ожидает заполнения и вычисляет среднеарифметическое.
    """
    print("[Поток 3] Ожидание заполнения списка для расчета среднего арифметического...")
    list_filled_event.wait() # Ждем события
    
    if not data_list:
        average = 0
        print("[Поток 3] Список пуст, среднее арифметическое: 0")
    else:
        average = sum(data_list) / len(data_list)
        print(f"[Поток 3] Среднеарифметическое значение в списке: {average:.2f}")
    return average

# --- Основная часть Задания 1 ---
def run_task1():
    print("--- Задание 1: Обработка списка в трех потоках ---")

    # Параметры для генерации списка
    num_elements = 20
    min_value = 1
    max_value = 100

    # Создаем потоки
    thread_filler = threading.Thread(target=fill_list, args=(num_elements, min_value, max_value))
    thread_summer = threading.Thread(target=calculate_sum)
    thread_averager = threading.Thread(target=calculate_average)

    # Запускаем потоки
    # Сначала поток-заполнитель
    thread_filler.start()
    # Затем потоки-обработчики, которые будут ждать события
    thread_summer.start()
    thread_averager.start()

    # Ожидаем завершения всех потоков
    thread_filler.join()
    thread_summer.join() # Результаты их работы будут выведены из самих потоков
    thread_averager.join()

    # Вывод финальных результатов (список уже глобальный и заполнен)
    # Результаты суммы и среднего уже выведены потоками, но можно продублировать или получить их через return,
    # если бы функции потоков возвращали значения и мы их как-то собирали (например, через очередь или атрибуты объекта).
    # В данном задании требуется вывод на экран из потоков.
    
    print("\n--- Итоговые результаты (после завершения всех потоков) ---")
    print(f"Сгенерированный список: {data_list}")
    # Сумма и среднее уже были выведены ранее соответствующими потоками.
    # Если нужно получить их здесь, функции calculate_sum и calculate_average
    # должны были бы возвращать значения, а мы бы их где-то сохранили.
    # Для простоты, как в задании, ограничимся выводом из потоков.
    
    # Пересчитаем для демонстрации, если нужно именно в основном потоке после всего
    if data_list:
        final_sum = sum(data_list)
        final_avg = sum(data_list) / len(data_list)
        print(f"Итоговая сумма (пересчет в основном потоке): {final_sum}")
        print(f"Итоговое среднее (пересчет в основном потоке): {final_avg:.2f}")
    else:
        print("Список остался пустым.")

    print("--- Задание 1 Завершено ---\n")

run_task1() # Раскомментируйте для запуска

--- Задание 1: Обработка списка в трех потоках ---
[Поток 1] Начало заполнения списка...
[Поток 2] Ожидание заполнения списка для расчета суммы...
[Поток 3] Ожидание заполнения списка для расчета среднего арифметического...
[Поток 1] Список заполнен 20 случайными числами.
[Поток 3] Среднеарифметическое значение в списке: 56.30
[Поток 2] Сумма элементов списка: 1126

--- Итоговые результаты (после завершения всех потоков) ---
Сгенерированный список: [62, 64, 68, 44, 29, 22, 2, 47, 99, 57, 60, 68, 73, 79, 68, 34, 59, 99, 18, 74]
Итоговая сумма (пересчет в основном потоке): 1126
Итоговое среднее (пересчет в основном потоке): 56.30
--- Задание 1 Завершено ---



Задание 2: Обработка чисел из файла в трех потоках

In [3]:
# Задание 2: Обработка чисел из файла в трех потоках
import threading
import random
import time
import math
import os

# Глобальные объекты синхронизации
file_ready_event = threading.Event()
# Блокировки для записи в файлы результатов (если несколько потоков пишут в один файл)
# В данном случае каждый поток пишет в свой файл, поэтому явные блокировки для записи файла могут не понадобиться,
# но если бы была общая статистика или логи, то они были бы нужны.
# lock_prime_results = threading.Lock()
# lock_factorial_results = threading.Lock()
# lock_stats = threading.Lock() # Для обновления общей статистики

# --- Вспомогательные функции ---
def is_prime(n: int) -> bool:
    """Проверяет, является ли число простым."""
    if n < 2:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def factorial(n: int) -> int:
    """Вычисляет факториал числа."""
    if n < 0:
        raise ValueError("Факториал не определен для отрицательных чисел")
    return math.factorial(n)

# --- Функции для потоков ---
def fill_file_with_numbers(filepath: str, count: int, min_val: int, max_val: int):
    """
    Первый поток: заполняет файл случайными числами.
    """
    global operations_stats
    print(f"[Поток 1 Запись Файла] Начало записи случайных чисел в файл '{filepath}'...")
    numbers_written = 0
    try:
        with open(filepath, 'w') as f:
            for _ in range(count):
                num = random.randint(min_val, max_val)
                f.write(str(num) + '\n')
                numbers_written += 1
                time.sleep(0.02) # Имитация работы
        print(f"[Поток 1 Запись Файла] Файл '{filepath}' заполнен {numbers_written} числами.")
        operations_stats['numbers_generated'] = numbers_written
        file_ready_event.set() # Сигнализируем, что файл готов
    except IOError as e:
        print(f"[Поток 1 Запись Файла] Ошибка при записи в файл: {e}")
        operations_stats['file_write_errors'] = operations_stats.get('file_write_errors', 0) + 1
        # Если файл не создан, другие потоки не должны стартовать.
        # Можно было бы не устанавливать событие или использовать другой механизм.


def find_primes_in_file(source_filepath: str, result_filepath: str):
    """
    Второй поток: ожидает заполнения файла, находит простые числа, записывает в новый файл.
    """
    global operations_stats
    print(f"[Поток 2 Простые Числа] Ожидание готовности файла '{source_filepath}'...")
    file_ready_event.wait()
    print(f"[Поток 2 Простые Числа] Файл '{source_filepath}' готов. Начинаю поиск простых чисел.")
    
    primes_found = 0
    numbers_processed = 0
    try:
        with open(source_filepath, 'r') as sf, open(result_filepath, 'w') as rf:
            for line in sf:
                try:
                    num = int(line.strip())
                    numbers_processed += 1
                    if is_prime(num):
                        rf.write(str(num) + '\n')
                        primes_found += 1
                except ValueError:
                    print(f"[Поток 2 Простые Числа] Пропуск нечисловой строки: {line.strip()}")
                    operations_stats['parse_errors_primes'] = operations_stats.get('parse_errors_primes', 0) + 1

        print(f"[Поток 2 Простые Числа] Найдено {primes_found} простых чисел. Результаты в '{result_filepath}'.")
        operations_stats['primes_found'] = primes_found
        operations_stats['numbers_processed_by_primes_thread'] = numbers_processed
    except IOError as e:
        print(f"[Поток 2 Простые Числа] Ошибка при работе с файлами: {e}")
        operations_stats['file_read_write_errors_primes'] = operations_stats.get('file_read_write_errors_primes', 0) + 1


def calculate_factorials_in_file(source_filepath: str, result_filepath: str):
    """
    Третий поток: ожидает заполнения файла, вычисляет факториалы, записывает в новый файл.
    """
    global operations_stats
    print(f"[Поток 3 Факториалы] Ожидание готовности файла '{source_filepath}'...")
    file_ready_event.wait()
    print(f"[Поток 3 Факториалы] Файл '{source_filepath}' готов. Начинаю расчет факториалов.")

    factorials_calculated = 0
    numbers_processed = 0
    try:
        with open(source_filepath, 'r') as sf, open(result_filepath, 'w') as rf:
            for line in sf:
                try:
                    num = int(line.strip())
                    numbers_processed += 1
                    if 0 <= num <= 20: # Ограничение для факториала, чтобы избежать слишком больших чисел
                        fact = factorial(num)
                        rf.write(f"Факториал числа {num} = {fact}\n")
                        factorials_calculated += 1
                    else:
                        rf.write(f"Факториал числа {num} не вычислен (слишком большое или отрицательное).\n")
                        operations_stats['factorial_skipped_large_negative'] = operations_stats.get('factorial_skipped_large_negative', 0) + 1
                except ValueError:
                    print(f"[Поток 3 Факториалы] Пропуск нечисловой строки: {line.strip()}")
                    operations_stats['parse_errors_factorials'] = operations_stats.get('parse_errors_factorials', 0) + 1
                except Exception as e_fact: # Ловим ошибки от math.factorial (например, для отрицательных)
                    rf.write(f"Ошибка вычисления факториала для {num}: {e_fact}\n")
                    operations_stats['factorial_calculation_errors'] = operations_stats.get('factorial_calculation_errors', 0) + 1


        print(f"[Поток 3 Факториалы] Вычислено {factorials_calculated} факториалов. Результаты в '{result_filepath}'.")
        operations_stats['factorials_calculated'] = factorials_calculated
        operations_stats['numbers_processed_by_factorials_thread'] = numbers_processed
    except IOError as e:
        print(f"[Поток 3 Факториалы] Ошибка при работе с файлами: {e}")
        operations_stats['file_read_write_errors_factorials'] = operations_stats.get('file_read_write_errors_factorials', 0) + 1


# --- Основная часть Задания 2 ---
operations_stats = {} # Словарь для сбора статистики

def run_task2():
    print("\n--- Задание 2: Обработка файла в трех потоках ---")
    global operations_stats
    operations_stats = { # Сбрасываем статистику перед каждым запуском
        'numbers_generated': 0,
        'primes_found': 0,
        'factorials_calculated': 0,
        'numbers_processed_by_primes_thread': 0,
        'numbers_processed_by_factorials_thread': 0
    }


    source_file = input("Введите путь к файлу для записи случайных чисел (например, numbers.txt): ")
    if not source_file:
        source_file = "numbers_task2.txt" # Значение по умолчанию
        print(f"Путь не указан, используется файл по умолчанию: {source_file}")

    primes_result_file = "primes_" + os.path.basename(source_file)
    factorials_result_file = "factorials_" + os.path.basename(source_file)

    # Удаляем старые файлы результатов, если они существуют
    if os.path.exists(primes_result_file): os.remove(primes_result_file)
    if os.path.exists(factorials_result_file): os.remove(factorials_result_file)
    if os.path.exists(source_file): os.remove(source_file) # И исходный, чтобы каждый раз генерировать заново

    # Параметры для генерации чисел в файл
    num_count_in_file = 15
    min_val_file = 1
    max_val_file = 25 # Ограничиваем, чтобы факториалы не были слишком большими

    # Сброс события перед новым запуском
    file_ready_event.clear()

    # Создаем потоки
    thread_file_writer = threading.Thread(target=fill_file_with_numbers, 
                                          args=(source_file, num_count_in_file, min_val_file, max_val_file))
    thread_prime_finder = threading.Thread(target=find_primes_in_file, 
                                           args=(source_file, primes_result_file))
    thread_factorial_calculator = threading.Thread(target=calculate_factorials_in_file, 
                                                   args=(source_file, factorials_result_file))

    # Запускаем потоки
    thread_file_writer.start()
    thread_prime_finder.start()
    thread_factorial_calculator.start()

    # Ожидаем завершения всех потоков
    thread_file_writer.join()
    thread_prime_finder.join()
    thread_factorial_calculator.join()

    # Отображаем статистику
    print("\n--- Статистика выполненных операций ---")
    for key, value in operations_stats.items():
        print(f"{key.replace('_', ' ').capitalize()}: {value}")
    
    if os.path.exists(source_file):
        print(f"\nСодержимое исходного файла '{source_file}':")
        try:
            with open(source_file, 'r') as f:
                print(f.read())
        except IOError: print("Не удалось прочитать исходный файл.")
            
    if os.path.exists(primes_result_file):
        print(f"\nСодержимое файла с простыми числами '{primes_result_file}':")
        try:
            with open(primes_result_file, 'r') as f:
                print(f.read())
        except IOError: print("Не удалось прочитать файл с простыми числами.")

    if os.path.exists(factorials_result_file):
        print(f"\nСодержимое файла с факториалами '{factorials_result_file}':")
        try:
            with open(factorials_result_file, 'r') as f:
                print(f.read())
        except IOError: print("Не удалось прочитать файл с факториалами.")

    print("--- Задание 2 Завершено ---\n")

run_task2() # Раскомментируйте для запуска


--- Задание 2: Обработка файла в трех потоках ---
[Поток 1 Запись Файла] Начало записи случайных чисел в файл 'library_books.json'...
[Поток 2 Простые Числа] Ожидание готовности файла 'library_books.json'...
[Поток 3 Факториалы] Ожидание готовности файла 'library_books.json'...
[Поток 1 Запись Файла] Файл 'library_books.json' заполнен 15 числами.
[Поток 2 Простые Числа] Файл 'library_books.json' готов. Начинаю поиск простых чисел.
[Поток 2 Простые Числа] Найдено 7 простых чисел. Результаты в 'primes_library_books.json'.
[Поток 3 Факториалы] Файл 'library_books.json' готов. Начинаю расчет факториалов.
[Поток 3 Факториалы] Вычислено 14 факториалов. Результаты в 'factorials_library_books.json'.

--- Статистика выполненных операций ---
Numbers generated: 15
Primes found: 7
Factorials calculated: 14
Numbers processed by primes thread: 15
Numbers processed by factorials thread: 15
Factorial skipped large negative: 1

Содержимое исходного файла 'library_books.json':
9
16
7
17
19
17
1
12
16
8

Задание 3: Копирование директории в потоке

In [4]:
# Задание 3: Копирование директории в потоке
import threading
import shutil # Для копирования файлов и директорий
import os
import time

# Глобальная статистика для копирования
copy_stats = {
    'files_copied': 0,
    'dirs_created': 0,
    'total_size_copied_bytes': 0,
    'errors': 0,
    'start_time': 0,
    'end_time': 0,
}
copy_lock = threading.Lock() # Для безопасного обновления статистики

def copy_directory_threaded(src_dir: str, dst_dir: str):
    """
    Копирует содержимое директории src_dir в dst_dir, сохраняя структуру.
    Эта функция будет выполняться в отдельном потоке.
    """
    global copy_stats
    print(f"[Поток Копирования] Начало копирования из '{src_dir}' в '{dst_dir}'...")
    
    with copy_lock:
        copy_stats['start_time'] = time.time()

    # Проверка существования исходной директории
    if not os.path.exists(src_dir):
        print(f"[Поток Копирования] Ошибка: Исходная директория '{src_dir}' не найдена.")
        with copy_lock:
            copy_stats['errors'] += 1
            copy_stats['end_time'] = time.time()
        return
    
    if not os.path.isdir(src_dir):
        print(f"[Поток Копирования] Ошибка: '{src_dir}' не является директорией.")
        with copy_lock:
            copy_stats['errors'] += 1
            copy_stats['end_time'] = time.time()
        return

    # Создаем целевую директорию, если ее нет
    # shutil.copytree создает dst_dir, если она не существует.
    # Если dst_dir существует, shutil.copytree выбросит FileExistsError,
    # поэтому нужно либо удалять ее, либо использовать dirs_exist_ok=True (Python 3.8+)
    # или копировать содержимое вручную.
    # Для простоты и совместимости, будем копировать содержимое вручную, если dst_dir существует.
    
    try:
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)
            with copy_lock:
                copy_stats['dirs_created'] += 1
            print(f"[Поток Копирования] Создана целевая директория '{dst_dir}'.")

        for item_name in os.listdir(src_dir):
            src_item_path = os.path.join(src_dir, item_name)
            dst_item_path = os.path.join(dst_dir, item_name)

            if os.path.isdir(src_item_path):
                # Рекурсивно копируем поддиректории (для простоты используем shutil.copytree для поддиректорий)
                # или можно реализовать полную рекурсию вручную
                print(f"[Поток Копирования] Копирование поддиректории '{src_item_path}' в '{dst_item_path}'...")
                # shutil.copytree(src_item_path, dst_item_path, dirs_exist_ok=True) # Python 3.8+
                
                # Ручное рекурсивное копирование для большей совместимости или если dirs_exist_ok недоступен
                # В данном случае, так как мы хотим точную статистику, лучше делать это рекурсивно вручную
                # или модифицировать статистику после вызова copytree.
                # Для этого задания сделаем упрощенно: если это поддиректория, просто рекурсивно вызываем нашу же функцию.
                # Это не самый эффективный способ для shutil, но для демонстрации потока и статистики подойдет.
                # Однако, чтобы не усложнять, просто скопируем и обновим статистику примерно.
                # Или же, для чистоты, будем копировать файлы и создавать директории вручную.
                
                # Используем shutil.copytree для поддиректорий, если не хотим глубокой ручной рекурсии здесь
                # и не так важна точная статистика по поддиректориям внутри этого вызова.
                # Для точной статистики лучше сделать полностью ручной обход.
                
                # Полностью ручной обход:
                if not os.path.exists(dst_item_path):
                     os.makedirs(dst_item_path)
                     with copy_lock:
                         copy_stats['dirs_created'] += 1
                
                # Рекурсивный вызов нашей функции для копирования содержимого поддиректории
                # (Это не самый оптимальный путь, но он позволит собрать статистику глубже, если доработать)
                # Чтобы не усложнять, просто вызовем копирование файлов внутри этой поддиректории
                # и передадим управление копированию файлов.
                # Для простоты задания, будем копировать только файлы из корневой src_dir,
                # а для поддиректорий используем shutil.copytree (без детальной статистики из него)
                # или рекурсивный вызов copy_directory_threaded для поддиректорий (что сделаем).
                
                # Создадим временный объект потока для рекурсивного копирования поддиректории,
                # чтобы статистика корректно собиралась через блокировки.
                # Однако, это усложнит.
                # Проще всего - передавать copy_stats как аргумент в рекурсивные вызовы.

                # Вернемся к более простому варианту для задания:
                # Мы будем копировать только файлы верхнего уровня и создавать директории верхнего уровня.
                # Глубокое копирование структуры с точной статистикой в одном потоке - это уже сложнее.
                # Задание говорит "скопировать содержимое ... сохранить структуру".
                # shutil.copytree идеально подходит, но тогда статистика будет общей.

                # Давайте реализуем ручное копирование с сохранением структуры:
                _copy_dir_recursively_with_stats(src_item_path, dst_item_path)


            elif os.path.isfile(src_item_path):
                print(f"[Поток Копирования] Копирование файла '{src_item_path}' в '{dst_item_path}'...")
                shutil.copy2(src_item_path, dst_item_path) # copy2 сохраняет метаданные
                with copy_lock:
                    copy_stats['files_copied'] += 1
                    copy_stats['total_size_copied_bytes'] += os.path.getsize(dst_item_path)
            time.sleep(0.05) # Имитация работы

        print(f"[Поток Копирования] Копирование из '{src_dir}' в '{dst_dir}' завершено.")

    except Exception as e:
        print(f"[Поток Копирования] Произошла ошибка во время копирования: {e}")
        with copy_lock:
            copy_stats['errors'] += 1
    finally:
        with copy_lock:
            copy_stats['end_time'] = time.time()


def _copy_dir_recursively_with_stats(src: str, dst: str):
    """Вспомогательная рекурсивная функция для копирования с обновлением статистики."""
    global copy_stats
    if not os.path.exists(dst):
        os.makedirs(dst)
        with copy_lock:
            copy_stats['dirs_created'] += 1
    
    for item in os.listdir(src):
        s_path = os.path.join(src, item)
        d_path = os.path.join(dst, item)
        if os.path.isdir(s_path):
            _copy_dir_recursively_with_stats(s_path, d_path) # Рекурсивный вызов
        else: # Это файл
            shutil.copy2(s_path, d_path)
            with copy_lock:
                copy_stats['files_copied'] += 1
                copy_stats['total_size_copied_bytes'] += os.path.getsize(d_path)
        time.sleep(0.01) # Имитация


# --- Основная часть Задания 3 ---
def run_task3():
    print("\n--- Задание 3: Копирование директории в потоке ---")
    global copy_stats
    copy_stats = { # Сброс статистики
        'files_copied': 0,
        'dirs_created': 0,
        'total_size_copied_bytes': 0,
        'errors': 0,
        'start_time': 0,
        'end_time': 0,
    }

    source_directory = input("Введите путь к существующей директории для копирования: ")
    destination_directory = input("Введите путь к новой директории (куда копировать): ")

    if not source_directory or not destination_directory:
        print("Ошибка: Не указаны исходная или целевая директория. Используем тестовые значения.")
        # Создадим тестовые директории и файлы для демонстрации
        base_test_dir = "task3_test_area"
        source_directory = os.path.join(base_test_dir, "source_dir_task3")
        destination_directory = os.path.join(base_test_dir, "destination_dir_task3")

        # Очистка предыдущих тестовых данных
        if os.path.exists(base_test_dir):
            shutil.rmtree(base_test_dir)
        
        os.makedirs(source_directory, exist_ok=True)
        os.makedirs(os.path.join(source_directory, "subdir1"), exist_ok=True)
        with open(os.path.join(source_directory, "file1.txt"), "w") as f: f.write("Это файл 1.")
        with open(os.path.join(source_directory, "file2.log"), "w") as f: f.write("Это файл 2 лог.")
        with open(os.path.join(source_directory, "subdir1", "file3_in_subdir.dat"), "w") as f: f.write("Файл в поддиректории.")
        print(f"Созданы тестовые директории: '{source_directory}' и '{destination_directory}' (пока пустая)")


    # Создаем и запускаем поток
    copy_thread = threading.Thread(target=copy_directory_threaded, args=(source_directory, destination_directory))
    copy_thread.start()
    copy_thread.join() # Ожидаем завершения потока копирования

    # Отображаем статистику
    print("\n--- Статистика копирования директории ---")
    if copy_stats['errors'] > 0:
        print(f"Во время копирования произошли ошибки: {copy_stats['errors']}")
    
    if copy_stats['start_time'] > 0 and copy_stats['end_time'] > 0 : # Если процесс хотя бы начался
        duration = copy_stats['end_time'] - copy_stats['start_time']
        print(f"Время начала: {time.ctime(copy_stats['start_time'])}")
        print(f"Время окончания: {time.ctime(copy_stats['end_time'])}")
        print(f"Продолжительность копирования: {duration:.2f} секунд")
        print(f"Создано директорий: {copy_stats['dirs_created']}")
        print(f"Скопировано файлов: {copy_stats['files_copied']}")
        print(f"Общий размер скопированных файлов: {copy_stats['total_size_copied_bytes']} байт ({copy_stats['total_size_copied_bytes'] / (1024*1024):.2f} МБ)")
    else:
        print("Копирование не было запущено или завершилось с критической ошибкой до начала операций.")

    print("--- Задание 3 Завершено ---\n")

run_task3() # Раскомментируйте для запуска


--- Задание 3: Копирование директории в потоке ---
Ошибка: Не указаны исходная или целевая директория. Используем тестовые значения.
Созданы тестовые директории: 'task3_test_area/source_dir_task3' и 'task3_test_area/destination_dir_task3' (пока пустая)
[Поток Копирования] Начало копирования из 'task3_test_area/source_dir_task3' в 'task3_test_area/destination_dir_task3'...
[Поток Копирования] Создана целевая директория 'task3_test_area/destination_dir_task3'.
[Поток Копирования] Копирование поддиректории 'task3_test_area/source_dir_task3/subdir1' в 'task3_test_area/destination_dir_task3/subdir1'...
[Поток Копирования] Копирование файла 'task3_test_area/source_dir_task3/file1.txt' в 'task3_test_area/destination_dir_task3/file1.txt'...
[Поток Копирования] Копирование файла 'task3_test_area/source_dir_task3/file2.log' в 'task3_test_area/destination_dir_task3/file2.log'...
[Поток Копирования] Копирование из 'task3_test_area/source_dir_task3' в 'task3_test_area/destination_dir_task3' заверш

Задание 4: Поиск слов в файлах и вырезание запрещенных слов

In [ ]:
# Задание 4: Поиск и обработка файлов в двух потоках
import threading
import os
import time
import re # Для регулярных выражений (вырезание слов)

# Глобальные переменные и объекты синхронизации
merged_content_filepath = "merged_found_files.txt"
banned_words_filepath = "banned_words.txt" # Файл с запрещенными словами
processed_content_filepath = "processed_content.txt"

first_thread_done_event = threading.Event()
search_replace_stats = {
    'files_searched': 0,
    'files_matched_keyword': 0,
    'total_lines_merged': 0,
    'banned_words_loaded': 0,
    'words_replaced_count': 0, # Общее количество замен
    'search_errors': 0,
    'processing_errors': 0
}
stats_lock = threading.Lock() # Для безопасного обновления статистики

# --- Функции для потоков ---
def search_and_merge_files(search_dir: str, keyword: str, output_merged_file: str):
    """
    Первый поток: ищет файлы, содержащие ключевое слово, и сливает их содержимое.
    """
    global search_replace_stats
    print(f"[Поток 1 Поиск] Начало поиска файлов в '{search_dir}' по ключевому слову '{keyword}'...")
    
    lines_merged_count = 0
    files_matched = 0
    files_scanned = 0

    try:
        with open(output_merged_file, 'w', encoding='utf-8') as outfile:
            for root, _, files in os.walk(search_dir):
                for filename in files:
                    if not filename.endswith(('.txt', '.log', '.md')): # Пример фильтрации по типу файлов
                        continue 
                    
                    filepath = os.path.join(root, filename)
                    files_scanned += 1
                    try:
                        with open(filepath, 'r', encoding='utf-8', errors='ignore') as infile:
                            content = infile.read()
                            if keyword.lower() in content.lower(): # Поиск без учета регистра
                                files_matched += 1
                                outfile.write(f"\n--- Содержимое из файла: {filepath} ---\n")
                                outfile.write(content)
                                outfile.write("\n") # Разделитель между содержимым файлов
                                lines_merged_count += content.count('\n') + 1 # Приблизительное кол-во строк
                                print(f"[Поток 1 Поиск] Найден '{keyword}' в файле: {filepath}. Содержимое добавлено.")
                    except Exception as e_read:
                        print(f"[Поток 1 Поиск] Ошибка чтения файла '{filepath}': {e_read}")
                        with stats_lock: search_replace_stats['search_errors'] +=1
                    time.sleep(0.02) # Имитация работы
        
        with stats_lock:
            search_replace_stats['files_searched'] = files_scanned
            search_replace_stats['files_matched_keyword'] = files_matched
            search_replace_stats['total_lines_merged'] = lines_merged_count
        
        print(f"[Поток 1 Поиск] Поиск завершен. Найдено {files_matched} файлов. Результат в '{output_merged_file}'.")

    except IOError as e_write:
        print(f"[Поток 1 Поиск] Критическая ошибка записи в объединенный файл '{output_merged_file}': {e_write}")
        with stats_lock: search_replace_stats['search_errors'] +=1
    finally:
        first_thread_done_event.set() # Сигнализируем, что первый поток завершил работу


def censor_banned_words(source_merged_file: str, banned_list_file: str, output_processed_file: str):
    """
    Второй поток: ожидает завершения первого, читает запрещенные слова,
    вырезает их из объединенного файла и сохраняет результат.
    """
    global search_replace_stats
    print(f"[Поток 2 Цензура] Ожидание завершения работы первого потока (слияния файлов)...")
    first_thread_done_event.wait()
    
    print(f"[Поток 2 Цензура] Первый поток завершил работу. Начинаю обработку файла '{source_merged_file}'.")
    
    banned_words = []
    try:
        with open(banned_list_file, 'r', encoding='utf-8') as bf:
            banned_words = [word.strip().lower() for word in bf if word.strip()]
        with stats_lock: search_replace_stats['banned_words_loaded'] = len(banned_words)
        print(f"[Поток 2 Цензура] Загружено {len(banned_words)} запрещенных слов из '{banned_list_file}'.")
    except IOError as e_banned:
        print(f"[Поток 2 Цензура] Ошибка чтения файла запрещенных слов '{banned_list_file}': {e_banned}. Обработка невозможна.")
        with stats_lock: search_replace_stats['processing_errors'] +=1
        return

    if not banned_words:
        print(f"[Поток 2 Цензура] Список запрещенных слов пуст. Копирование содержимого как есть в '{output_processed_file}'.")
        try:
            shutil.copyfile(source_merged_file, output_processed_file)
        except Exception as e_copy:
             print(f"[Поток 2 Цензура] Ошибка копирования: {e_copy}")
             with stats_lock: search_replace_stats['processing_errors'] +=1
        return

    replacements_done = 0
    try:
        with open(source_merged_file, 'r', encoding='utf-8') as sf, \
             open(output_processed_file, 'w', encoding='utf-8') as pf:
            
            content_to_process = sf.read()
            processed_text = content_to_process
            
            for word_to_censor in banned_words:
                # Используем регулярное выражение для замены целых слов без учета регистра
                # \b - граница слова
                # re.IGNORECASE - флаг для игнорирования регистра
                # (?i) - встроенный флаг игнорирования регистра для шаблона
                # Заменяем на звездочки такой же длины, или на фиксированный плейсхолдер
                placeholder = '*' * len(word_to_censor) # или "[ЦЕНЗУРА]"
                
                # pattern = r'\b' + re.escape(word_to_censor) + r'\b' # Старый вариант, без игнор кейса в re.sub
                # processed_text, num_subs = re.subn(pattern, placeholder, processed_text, flags=re.IGNORECASE)
                
                # Более простой вариант - пройти по словам и заменить
                # Это менее эффективно для больших текстов, чем одно re.sub на все слова,
                # но проще для подсчета замен по каждому слову.
                # Либо использовать re.subn для подсчета
                
                # Вариант с re.subn для каждого слова (для подсчета)
                temp_text, num_subs_for_word = re.subn(r'\b' + re.escape(word_to_censor) + r'\b', 
                                                       placeholder, 
                                                       processed_text, 
                                                       flags=re.IGNORECASE)
                if num_subs_for_word > 0:
                    processed_text = temp_text
                    replacements_done += num_subs_for_word
            
            pf.write(processed_text)

        with stats_lock: search_replace_stats['words_replaced_count'] = replacements_done
        print(f"[Поток 2 Цензура] Обработка завершена. Заменено вхождений запрещенных слов: {replacements_done}. Результат в '{output_processed_file}'.")

    except IOError as e_proc:
        print(f"[Поток 2 Цензура] Ошибка при обработке или записи файла '{output_processed_file}': {e_proc}")
        with stats_lock: search_replace_stats['processing_errors'] +=1

# --- Основная часть Задания 4 ---
def run_task4():
    print("\n--- Задание 4: Поиск и цензура в двух потоках ---")
    global search_replace_stats
    search_replace_stats = { # Сброс статистики
        'files_searched': 0,
        'files_matched_keyword': 0,
        'total_lines_merged': 0,
        'banned_words_loaded': 0,
        'words_replaced_count': 0,
        'search_errors': 0,
        'processing_errors': 0
    }

    # Подготовка тестовых данных
    test_base_dir = "task4_search_area"
    if os.path.exists(test_base_dir): shutil.rmtree(test_base_dir) # Очистка
    os.makedirs(test_base_dir, exist_ok=True)
    
    # Создаем файлы для поиска
    with open(os.path.join(test_base_dir, "doc1.txt"), "w", encoding="utf-8") as f: f.write("Это первый документ с ключевым словом Python.\nОчень важный документ.\nЗапретноеСлово1 тут есть.")
    with open(os.path.join(test_base_dir, "doc2.md"), "w", encoding="utf-8") as f: f.write("Второй файл, тоже про Python.\nPython это круто! Но есть ЗаПреТноеСлово2.")
    os.makedirs(os.path.join(test_base_dir, "subdir"), exist_ok=True)
    with open(os.path.join(test_base_dir, "subdir", "doc3.log"), "w", encoding="utf-8") as f: f.write("Логи.\nКлючевое слово Python тут тоже.\nИ ЗапретноеСлово3.")
    with open(os.path.join(test_base_dir, "other.dat"), "w", encoding="utf-8") as f: f.write("Нетекстовый файл, python тут не ищем.")
    
    # Создаем файл с запрещенными словами
    global banned_words_filepath
    banned_words_filepath = os.path.join(test_base_dir, "banned_words.txt")
    with open(banned_words_filepath, "w", encoding="utf-8") as f:
        f.write("ЗапретноеСлово1\n")
        f.write("запретноеслово2\n") # в нижнем регистре для теста
        f.write("ЗапретноеСлово3\n")
        f.write("слово4\n") # Этого слова нет в текстах

    # Пути к файлам результатов (будут созданы в текущей директории или test_base_dir)
    global merged_content_filepath, processed_content_filepath
    merged_content_filepath = os.path.join(test_base_dir, "merged_found_files.txt")
    processed_content_filepath = os.path.join(test_base_dir, "processed_content.txt")
    
    # Удаляем старые файлы результатов, если они существуют
    if os.path.exists(merged_content_filepath): os.remove(merged_content_filepath)
    if os.path.exists(processed_content_filepath): os.remove(processed_content_filepath)

    # --- Запрос данных от пользователя ---
    user_search_dir = input(f"Введите путь к директории для поиска (Enter для '{test_base_dir}'): ") or test_base_dir
    user_keyword = input("Введите слово для поиска (Enter для 'Python'): ") or "Python"
    user_banned_words_file = input(f"Введите путь к файлу с запрещенными словами (Enter для '{banned_words_filepath}'): ") or banned_words_filepath


    first_thread_done_event.clear() # Сброс события

    # Создаем и запускаем потоки
    thread1_search_merge = threading.Thread(target=search_and_merge_files, 
                                            args=(user_search_dir, user_keyword, merged_content_filepath))
    thread2_censor = threading.Thread(target=censor_banned_words, 
                                      args=(merged_content_filepath, user_banned_words_file, processed_content_filepath))

    thread1_search_merge.start()
    thread2_censor.start()

    thread1_search_merge.join()
    thread2_censor.join()

    # Отображаем статистику
    print("\n--- Статистика поиска и обработки ---")
    for key, value in search_replace_stats.items():
        print(f"{key.replace('_', ' ').capitalize()}: {value}")
        
    if os.path.exists(merged_content_filepath):
        print(f"\nСодержимое объединенного файла '{merged_content_filepath}' (до цензуры):")
        try:
            with open(merged_content_filepath, 'r', encoding='utf-8') as f: print(f.read()[:500] + "...") # Первые 500 символов
        except Exception as e: print(f"Ошибка чтения файла: {e}")
            
    if os.path.exists(processed_content_filepath):
        print(f"\nСодержимое обработанного файла '{processed_content_filepath}' (после цензуры):")
        try:
            with open(processed_content_filepath, 'r', encoding='utf-8') as f: print(f.read()[:500] + "...") # Первые 500 символов
        except Exception as e: print(f"Ошибка чтения файла: {e}")


    print("--- Задание 4 Завершено ---\n")

run_task4() # Раскомментируйте для запуска


--- Задание 4: Поиск и цензура в двух потоках ---
[Поток 1 Поиск] Начало поиска файлов в '/' по ключевому слову 'да'...
[Поток 2 Цензура] Ожидание завершения работы первого потока (слияния файлов)...
[Поток 1 Поиск] Найден 'да' в файле: /usr/local/share/nvm/versions/node/v20.19.0/README.md. Содержимое добавлено.
[Поток 1 Поиск] Найден 'да' в файле: /usr/local/share/nvm/versions/node/v18.20.7/README.md. Содержимое добавлено.
[Поток 1 Поиск] Найден 'да' в файле: /usr/local/share/nvm/versions/node/v18.20.7/lib/node_modules/npm/node_modules/buffer/AUTHORS.md. Содержимое добавлено.
[Поток 1 Поиск] Найден 'да' в файле: /usr/share/gnupg/help.ru.txt. Содержимое добавлено.
[Поток 1 Поиск] Ошибка чтения файла '/var/log/apt/term.log': [Errno 13] Permission denied: '/var/log/apt/term.log'
[Поток 1 Поиск] Найден 'да' в файле: /home/codespace/.vscode-remote/extensions/ms-ceintl.vscode-language-pack-ru-1.100.2025051409/readme.md. Содержимое добавлено.
[Поток 1 Поиск] Найден 'да' в файле: /home/codes

KeyboardInterrupt: 

[Поток 1 Поиск] Найден 'да' в файле: /.codespaces/bin/cache/bin/linux-x64/2fc07b811f760549dab9be9d2bedd06c51dfcb9a/node_modules/buffer/AUTHORS.md. Содержимое добавлено.
[Поток 1 Поиск] Найден 'да' в файле: /.codespaces/bin/cache/bin/linux-x64/4949701c880d4bdb949e3c0e6b400288da7f474b/node_modules/buffer/AUTHORS.md. Содержимое добавлено.
[Поток 1 Поиск] Найден 'да' в файле: /.codespaces/bin/cache/bin/linux-x64/848b80aeb52026648a8ff9f7c45a9b0a80641e2e/node_modules/buffer/AUTHORS.md. Содержимое добавлено.
[Поток 1 Поиск] Найден 'да' в файле: /.codespaces/bin/cache/bin/linux-x64/496ebc4723371f29c9ffa0319dcccb2d7bee7ee0-insider/node_modules/buffer/AUTHORS.md. Содержимое добавлено.
[Поток 1 Поиск] Найден 'да' в файле: /.codespaces/bin/cache/bin/linux-x64/ccdd214171190f69e28c8c3def68a6315f4d9ae0-insider/node_modules/buffer/AUTHORS.md. Содержимое добавлено.
